# RAG - How to query

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])
print(OPENAI_API_KEY[:10])

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY,
    "X-OpenAI-BaseURL": OPENAI_URL
  }
)

print(client.is_ready())

### Start with (R) - Retrieval

In [ ]:
wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="How do planes fly",
    limit=5,
    return_properties=["text", "title"]
)

for item in response.objects:
    print(item.properties)

### Add (AG) - augmented generation - to make full RAG

#### Single Prompt

> Generate a response per **retrieved** object.

In [ ]:
# Let's add some colour to our lives :)
BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

In [ ]:
from weaviate.classes.generate import GenerativeConfig

wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,

    # TODO: add GenerativeConfig with OpenAI and "gpt-4o-mini"
    generative_provider=GenerativeConfig.ollama(
        model="qwen2.5:0.5b",
        api_endpoint=OPENAI_URL,
    ),
    
    # TODO: add a single prompt "Explain what this is about? {text}"
    single_prompt="Explain what this is about? {text}"
)

for item in response.objects:
    print(f"{BLUE}=== Source ===")
    print(item.properties)

    print(f"{PURPLE}=== Generated Response ===")
    print(item.generative.text)
    print("\n")

#### Grouped Task

> Generate one response based on all **retrieved** objects.

In [ ]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,

    generative_provider=GenerativeConfig.ollama(
        model="qwen2.5:0.5b",
        api_endpoint=OPENAI_URL,
    ),

    grouped_task="Explain, how do planes fly? Please only use the provided content."
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

#### Specify which properties to use for grouped task

In [ ]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    auto_limit=1,
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
    grouped_properties=["text", "title"],

    generative_provider=GenerativeConfig.ollama(
        model="qwen2.5:0.5b",
        api_endpoint=OPENAI_URL
    ),
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

## Set default Generative model

In [ ]:
from weaviate.classes.config import Reconfigure

wiki = client.collections.get("Wiki")

wiki.config.update(
    generative_config=Reconfigure.Generative.ollama(
        model="qwen2.5:0.5b",
        api_endpoint=OPENAI_URL
    )
)

> Try generative query without providing the model

In [ ]:
response = wiki.generate.near_text(
    query="What african animals do we have info on. Please only list those provided in here.",
    auto_limit=1,
    
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

## Close the client

In [ ]:
client.close()